# Knowledge Graph RAG Query Engine

With LlamaIndex, we could build Knowledge Graphs leveraging its rich data source ecosystem, with LLM or even [local models](https://colab.research.google.com/drive/1G6pcR0pXvSkdMQlAK_P-IrYgo-_staxd?usp=sharing) best practices.

And on top of that, we could already:
- Perform Graph RAG, or [Graph + Vector RAG](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html) with KnowledgeGraphIndex and GraphStore by building Knowledge Graphs with LlamaIndex.
- Perform text2cypher with KnowledgeGraphQueryEngine and GraphStore on existing Knowledge Graphs or those built with LlamaIndex.

Now, with KnowledgeGraphRAGQueryEngine and GraphStore, we could perform Graph RAG on existing Knowledge Graphs, too.

Let's first get ready for basic preparation of Llama Index.

In [ ]:
# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = "INSERT OPENAI KEY"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import OpenAI

from IPython.display import Markdown, display


# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="text-davinci-002")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size_limit=512)

In [ ]:
# For Azure OpenAI
import os
import json
import openai
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import LangChainLLM

import logging
import sys

from IPython.display import Markdown, display

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

openai.api_type = "azure"
openai.api_base = "INSERT AZURE API BASE"
openai.api_version = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "INSERT OPENAI KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")

lc_llm = AzureOpenAI(
    engine="INSERT DEPLOYMENT NAME",
    temperature=0,
    model="gpt-35-turbo",
)
llm = LangChainLLM(lc_llm)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="INSERT DEPLOYMENT NAME",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)

## Prepare for NebulaGraph

Before next step to perform Graph RAG on existing KG, let's ensure we have a running NebulaGraph with defined data schema.

In [ ]:
# Create a NebulaGraph (version 3.5.0 or newer) cluster with:
# Option 0 for machines with Docker: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
# Option 1 for Desktop: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext

# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE llamaindex;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# :sleep 10;
# CREATE TAG INDEX entity_index ON entity(name(256));

%pip install ipython-ngql nebula3-python

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "<password>"  # default is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

Prepare for StorageContext with graph_store as NebulaGraphStore

In [ ]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

Here, we assumed to have the same Knowledge Graph from [this turtorial](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html#optional-build-the-knowledge-graph-with-llamaindex)

## Perform Graph RAG Query

Finally, let's demo how to do Graph RAG towards an existing Knowledge Graph.

Here, we will leverage the `KnowledgeGraphQueryEngine`, with `NebulaGraphStore` as the `storage_context.graph_store`.

In [ ]:
from llama_index.query_engine import KnowledgeGraphRAGQueryEngine

query_engine = KnowledgeGraphRAGQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

In [ ]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.query_engine.knowledge_graph_query_engine:Graph Store Query: MATCH (p:`entity`)-[:relationship]->(e:`entity`) WHERE p.`entity`.`name` == 'Peter Quill' RETURN e.`entity`.`name`;
Final Response: 
Peter Quill is a character associated with the Guardians of the Galaxy. He has an alternate version of Gamora as a grandfather and is part of the Guardians of the Galaxy.


<b>
Peter Quill is a character associated with the Guardians of the Galaxy. He has an alternate version of Gamora as a grandfather and is part of the Guardians of the Galaxy.</b>

Optionally, we could choose to synthesise answer from two retrieved context from Knowledge Graph:
- Graph RAG, the default retrieval method, which extracts subgraph that's related to the key entities in the question.
- NL2GraphQuery, generate Knowledge Graph Query based on query and the Schema of the Knowledge Graph, which is by default switched off. We need to set `with_nl2graphquery=True` to enable it.

In [21]:
query_engine_with_nl2graphquery = KnowledgeGraphRAGQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
    with_nl2graphquery=True,
)

response = query_engine_with_nl2graphquery.query(
    "Tell me about James Gunn?",
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.query_engine.knowledge_graph_query_engine:
Graph Store Query: MATCH (e:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e.`entity`.`name` == 'James Gunn'
RETURN e2.`entity`.`name`
INFO:llama_index.query_engine.knowledge_graph_query_engine:
Graph Store Response: {'e2.entity.name': ['Guardians of the Galaxy', 'having basic story for Guardians', 'Disney', 'Marvel', 'Guardians of the Galaxy', 'controversial topics', 'pedophilia', 'rape']}
Final Response: James Gunn is a filmmaker who has written and directed the Guardians of the Galaxy films. He has also been involved in controversial topics such as pedophilia and rape. Gunn was fired by Marvel but later rehired as the director of the Guardians of the Galaxy film. He has also worked with Disney and Marvel on various projects.


<b>James Gunn is a filmmaker who has written and directed the Guardians of the Galaxy films. He has also been involved in controversial topics such as pedophilia and rape. Gunn was fired by Marvel but later rehired as the director of the Guardians of the Galaxy film. He has also worked with Disney and Marvel on various projects.</b>

And let's check the response's metadata to see:

- The Graph RAG Generate a Cypher Query towards the answer:

```cypher
Graph Store Query: MATCH (e:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e.`entity`.`name` == 'James Gunn'
RETURN e2.`entity`.`name`
```
- And also get subgraph of 'James Gunn'.

Finally, it combined the two nodes of context, to systematize the answer.

In [22]:
response.metadata

{'f08e6265-11e7-4bd8-a538-349e558132d1': {'query_str': 'Tell me about James Gunn?',
  'graph_store_query': "MATCH (e:`entity`)-[r:`relationship`]->(e2:`entity`)\nWHERE e.`entity`.`name` == 'James Gunn'\nRETURN e2.`entity`.`name`",
  'graph_store_response': {'e2.entity.name': ['Guardians of the Galaxy',
    'having basic story for Guardians',
    'Disney',
    'Marvel',
    'Guardians of the Galaxy',
    'controversial topics',
    'pedophilia',
    'rape']},
  'graph_schema': "Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]\nEdge properties: [{'edge': 'relationship', 'properties': [('relationship', 'string')]}]\nRelationships: ['(:entity)-[:relationship]->(:entity)']\n"},
 '89ebb1ff-7039-4fce-9bc3-4370cf314213': {'kg_rel_text': ['James James, Twitter feed, indefensible',
   'Gunn Gunn, publicly revealed to have been rehired as director of, film, never met with or considered any other director for, film',
   'Gunn Gunn, publicly revealed to have been rehired as 